In [1]:
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def finder_web_scrape(url, affiliate, wait_time_sec) :
# Initiate web driver
    driver = webdriver.Chrome()
#     url = 'https://www.finder.com.au/savings-accounts/best-savings-accounts'
    driver.get(url)

# Scroll down-up to load elements
    time.sleep(wait_time_sec)

    driver.execute_script("window.scrollTo(0, 2080)")

    time.sleep(wait_time_sec)

    driver.execute_script("window.scrollTo(0, 1080)")

# Set values for today, affiliate, and rank
    get_today = datetime.datetime.now()
    today = get_today.strftime('%d/%m/%Y')
#     affiliate = 'Finder'
    rank = 1

# Create list to be used as df column headers
#     results = [['Date', 'Affiliate', 'Position', 'Provider', 'Product', 'Maximum Rate', 'Standard Rate', 'Max Rate Conditions', 'Savings Details']]

# Get number of savings listing cards for range
    savings = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="comparison-table-0000000000"]/tbody//tr')))

# Click on all 'View Details' collapsible elements to load savings_details text element
    for i in range(1, len(savings) + 1):
        viewClass = f'/html/body/div[1]/div/div[5]/div[2]/div[2]/div[4]/section/form/table/tbody/tr[{i}]/td[8]/div//a//span'
        driver.execute_script('arguments[0].click();', WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, viewClass))))
    
# Additional loading time for slow-loading collapsible elements
    time.sleep(wait_time_sec)

# Get savings listing values and push to results list
    for i in range(1, len(savings) + 1):
        savings_listing = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="comparison-table-0000000000"]/tbody/tr[{i}]//a[1]'))).text.split(' ', 1)
        savings_max_rate = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="comparison-table-0000000000"]/tbody/tr[{i}]/td[3]/div//span'))).text
        savings_standard_rate = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="comparison-table-0000000000"]/tbody/tr[{i}]/td[4]/div/span'))).text
#     savings_intro_period = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="comparison-table-0000000000"]/tbody/tr[{i}]/td[5]/div'))).text
        savings_conditions = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="comparison-table-0000000000"]/tbody/tr[{i}]/td[7]/div/div[1]'))).text.replace('\n', ', ')
    
        savings_details = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="comparison-table-0000000000"]/tbody/tr[{i}]//p'))).text
    
        if savings_listing[0]=='Bank' :
            savings_entry = f'{today} | {affiliate} | {rank} | Bank of Queensland | Bank {savings_listing[1]} | {savings_max_rate} | {savings_standard_rate} | {savings_conditions} | {savings_details}'
        elif savings_listing[0]=='Commonwealth' :
            savings_entry = f'{today} | {affiliate} | {rank} | Commonwealth Bank | {savings_listing[0]} {savings_listing[1]} | {savings_max_rate} | {savings_standard_rate} | {savings_conditions} | {savings_details}'
        else :
            savings_entry = f'{today} | {affiliate} | {rank} | {savings_listing[0]} | {savings_listing[0]} {savings_listing[1]} | {savings_max_rate} | {savings_standard_rate} | {savings_conditions} | {savings_details}'
        rank = rank + 1
        results.append(savings_entry.split('|'))

# Close web driver
    driver.quit()

In [3]:
def mozo_web_scrape(url, affiliate, wait_time_sec) :
    driver = webdriver.Chrome()
#     url = 'https://mozo.com.au/savings-accounts'
    driver.get(url)

    time.sleep(wait_time_sec)

    driver.execute_script("window.scrollTo(0, 2080)")

    time.sleep(wait_time_sec)

    driver.execute_script("window.scrollTo(0, 1080)")

    get_today = datetime.datetime.now()
    today = get_today.strftime('%d/%m/%Y')
#     affiliate = 'Mozo'
    rank = 1

#     results = [['Date', 'Affiliate', 'Position', 'Provider', 'Product', 'Maximum Rate', 'Standard Rate', 'Max Rate Conditions', 'Savings Details']]

    savings = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="compare"]/div[2]/div[2]/ul/li')))

    for i in range(1, len(savings) + 1):
        viewClass = f'/html/body/div[3]/div[2]/div[2]/ul/li[{i}]/div[5]/div[3]/span[1]'
        driver.execute_script('arguments[0].click();', WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, viewClass))))
    
    time.sleep(wait_time_sec)

    for i in range(1, len(savings) + 1):
        savings_listing = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="compare"]/div[2]/div[2]/ul/li[{i}]//img'))).get_attribute('alt').replace('logo','')
        savings_product = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="compare"]/div[2]/div[2]/ul/li[{i}]/div[3]/h6'))).text
        # Take savings_max_rate[0]
        savings_max_rate = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="compare"]/div[2]/div[2]/ul/li[{i}]/div[4]/div[1]'))).text.split(' ')
        # Take savings_standard_rate[0]
        savings_standard_rate = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="compare"]/div[2]/div[2]/ul/li[{i}]/div[4]/div[2]'))).text.split(' ')
        savings_conditions = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="compare"]/div[2]/div[2]/ul/li[{i}]/div[6]/div[3]/dl[2]/dd'))).text.replace('\n', ', ')

        savings_details = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="compare"]/div[2]/div[2]/ul/li[{i}]//p'))).text

        savings_entry = savings_entry = f'{today} | {affiliate} | {rank} | {savings_listing} | {savings_product} | {savings_max_rate[0]} | {savings_standard_rate[0]} | {savings_conditions} | {savings_details}'
        rank = rank + 1
        results.append(savings_entry.split('|'))

    driver.quit()

In [7]:
def ratecity_web_scrape(url, affiliate, wait_time_sec) :
    driver = webdriver.Chrome()
#     url = 'https://www.ratecity.com.au/savings-accounts'
    driver.get(url)

    time.sleep(wait_time_sec)

    driver.execute_script("window.scrollTo(0, 1080)")

    time.sleep(wait_time_sec)

    driver.execute_script("window.scrollTo(0, 2080)")

    get_today = datetime.datetime.now()
    today = get_today.strftime('%d/%m/%Y')
#     affiliate = 'RateCity'
    rank = 1

#     results = [['Date', 'Affiliate', 'Position', 'Provider', 'Product', 'Maximum Rate', 'Standard Rate', 'Max Rate Conditions', 'Savings Details']]

    load_more_button = f'//*[@id="__next"]/div/main/div[3]/div/div[2]/div[4]/div[1]/div/button'
    load_more_clicks = 2

    for i in range(1, load_more_clicks):
        driver.execute_script('arguments[0].click();', WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, load_more_button))))
        time.sleep(wait_time_sec)
    
    read_more = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="rc-ratetable"]/div/div//span[contains(text()," ...read more")]')))
    for i in range(1, len(read_more) + 1):
        read_more_button = f'//*[@id="rc-ratetable"]/div/div//span[contains(text()," ...read more")]'
        driver.execute_script('arguments[0].click();', WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, read_more_button))))
        time.sleep(1.5)
    
    savings = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="rc-ratetable"]/div/div')))
    
    for i in range(1, len(savings) + 1) :
        savings_listing = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="rc-ratetable"]/div/div[{i}]/div[2]/div[1]/div/div/div[2]/div/img'))).get_attribute('alt')      
        savings_product = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="rc-ratetable"]/div/div[{i}]/div[2]/div[2]/div[1]/div/div[2]/p'))).text
        savings_max_rate = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="rc-ratetable"]/div/div[{i}]/div[3]/div[2]/div[1]/div[2]/div[1]/div/div[1]/p[1]'))).text.replace('*','')
        savings_standard_rate = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="rc-ratetable"]/div/div[{i}]/div[3]/div[2]/div[1]/div[2]/div[2]/div/div[1]/p[1]'))).text.replace('*','')
    
        try :
            savings_conditions = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="rc-ratetable"]/div/div[{i}]/div[2]/div[2]/div[4]/p'))).text.replace(' ...read less','')
        except :
            savings_conditions = 'No max rate conditions listed'
    
        savings_details = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="rc-ratetable"]/div/div[{i}]/div[2]/div[2]/div[4]'))).text.replace(' ...read less','').replace('\n',' ').replace('Special','Special: ')  
    
        if savings_details == '' :
            savings_entry = f'{today} | {affiliate} | {rank} | {savings_listing} | {savings_product} | {savings_max_rate} | {savings_standard_rate} | {savings_conditions} | No listed offer or details'
        else:
            savings_entry = f'{today} | {affiliate} | {rank} | {savings_listing} | {savings_product} | {savings_max_rate} | {savings_standard_rate} | {savings_conditions} | {savings_details}'
    
        rank = rank + 1
        results.append(savings_entry.split('|'))

    driver.quit()

In [5]:
def savings_web_scrape(url, affiliate, wait_time_sec) :
    driver = webdriver.Chrome()
#     url = 'https://www.savings.com.au/savings-accounts/'
    driver.get(url)

    time.sleep(wait_time_sec)

    driver.execute_script("window.scrollTo(0, 2080)")

    time.sleep(wait_time_sec)

    driver.execute_script("window.scrollTo(0, 1080)")

    get_today = datetime.datetime.now()
    today = get_today.strftime('%d/%m/%Y')
#     affiliate = 'Savings.com.au'
    rank = 1

#     results = [['Date', 'Affiliate', 'Position', 'Provider', 'Product', 'Maximum Rate', 'Standard Rate', 'Max Rate Conditions', 'Savings Details']]

    savings_listings = []
    savings_products = []
    savings_max_rates = []
    savings_standard_rates = []
    savings_conditions0 =[]
    savings_conditions1 = []
    savings_details = []
    results1 = [savings_listings, savings_products, savings_max_rates, savings_standard_rates, savings_conditions0, savings_conditions1, savings_details]

    savings = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="sca-table-wrapper-5f7e8ffd82091181b2ef6115"]/table/tbody//tr[@class="product-data"]')))

    for i in range(0, len(savings)) :   
        savings_listing = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="brand-logo-{i}"]/img'))).get_attribute('alt')
        savings_listings.append(savings_listing)
    
    for i in range(2, (len(savings) * 3), 3) :
        savings_product = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="sca-table-wrapper-5f7e8ffd82091181b2ef6115"]/table/tbody/tr[{i}]/td[2]/div/a'))).text
        savings_detail = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="sca-table-wrapper-5f7e8ffd82091181b2ef6115"]/table/tbody/tr[{i}]/td[2]/div//ul[@class="feature-list"]'))).text
        savings_products.append(savings_product)
        savings_details.append(savings_detail.replace('\n', ', '))
    
    for i in range(1, (len(savings) * 3), 3) :
        savings_max_rate = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="sca-table-wrapper-5f7e8ffd82091181b2ef6115"]/table/tbody/tr[{i}]/td[3]/app-sca-comparison-field/label/span'))).text
        savings_condition0 = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="sca-table-wrapper-5f7e8ffd82091181b2ef6115"]/table/tbody/tr[{i}]/td[3]/app-sca-comparison-field/label/div'))).text.replace('monthsthen', 'months then')
        savings_condition1 = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="sca-table-wrapper-5f7e8ffd82091181b2ef6115"]/table/tbody/tr[{i}]/td[2]/app-sca-comparison-field/label/div'))).text.replace('Conditions', ' Conditions')
        savings_standard_rate = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="sca-table-wrapper-5f7e8ffd82091181b2ef6115"]/table/tbody/tr[{i}]/td[2]/app-sca-comparison-field/label/span'))).text
        savings_max_rates.append(savings_max_rate)
        savings_standard_rates.append(savings_standard_rate)
        savings_conditions0.append(savings_condition0)
        savings_conditions1.append(savings_condition1)

    for i in range(0, len(savings)) : 
        savings_entry = f'{today} | {affiliate} | {rank} | {results1[0][i]} | {results1[1][i]} | {results1[2][i]}% | {results1[3][i]}% | {results1[5][i]} | {results1[4][i]}, {results1[6][i]}'
    
        rank = rank + 1
        results.append(savings_entry.split('|'))

    driver.quit()

In [8]:
bot_wait_time = 2.5
results = [['Date', 'Affiliate', 'Position', 'Provider', 'Product', 'Maximum Rate', 'Standard Rate', 'Max Rate Conditions', 'Savings Details']]

get_today = datetime.datetime.now()
today = get_today.strftime('%d-%m-%Y')
    
finder_web_scrape('https://www.finder.com.au/savings-accounts/best-savings-accounts', 'Finder', bot_wait_time)
mozo_web_scrape('https://mozo.com.au/savings-accounts', 'Mozo', bot_wait_time)
ratecity_web_scrape('https://www.ratecity.com.au/savings-accounts', 'RateCity', bot_wait_time)
savings_web_scrape('https://www.savings.com.au/savings-accounts/', 'Savings.com.au', bot_wait_time)

df = pd.DataFrame(results[1:], columns=results[0]).to_dict('list')
df2 = pd.DataFrame(df)

print(df2)
df2.to_csv(f'{today}EDBaffiliates_all.csv', index=False, encoding='utf-8')

           Date         Affiliate Position  \
0   26/12/2022            Finder        1    
1   26/12/2022            Finder        2    
2   26/12/2022            Finder        3    
3   26/12/2022            Finder        4    
4   26/12/2022            Finder        5    
5   26/12/2022            Finder        6    
6   26/12/2022            Finder        7    
7   26/12/2022            Finder        8    
8   26/12/2022            Finder        9    
9   26/12/2022            Finder       10    
10  26/12/2022            Finder       11    
11  26/12/2022            Finder       12    
12  26/12/2022              Mozo        1    
13  26/12/2022              Mozo        2    
14  26/12/2022              Mozo        3    
15  26/12/2022              Mozo        4    
16  26/12/2022              Mozo        5    
17  26/12/2022              Mozo        6    
18  26/12/2022              Mozo        7    
19  26/12/2022              Mozo        8    
20  26/12/2022          RateCity  